# ODEs: Projectile motion


In one of our problem sets, we simulated the the equation for a projectile. Of course, the answer is:

$$
\vec{x}(t) = \frac{1}{2}\vec{a}_0t^2 + \vec{v}_0 t + \vec{x}_0
$$

However, we now will be looking into direct solution of the ordinary differential equation using numerical methods. The equation of motion is

$$
\ddot{\vec{r}}(t) = -g\hat{y}
$$

We will try three different methods:

- Euler 
$$\vec{v}_{n+1} = \vec{v}_n + \tau \vec{a}_n$$
$$\vec{x}_{n+1} = \vec{x}_n + \tau \vec{v}_n$$
- Euler-Cromer
$$\vec{v}_{n+1} = \vec{v}_n + \tau \vec{a}_n$$
$$\vec{x}_{n+1} = \vec{x}_n + \tau \vec{v}_{n+1}$$
- Midpoint
$$\vec{v}_{n+1} = \vec{v}_n + \tau \vec{a}_n$$
$$\vec{x}_{n+1} = \vec{x}_n + \frac{\tau}{2}\left(\vec{v}_n + \vec{v}_{n+1}\right)$$


We will also investigate using air resistance:

$$F_D = \frac{1}{2}\rho v^2 C_D A$$


In [ ]:
import numpy as np
import matplotlib
matplotlib.rcParams['legend.fancybox'] = True
import matplotlib.pyplot as plt
from odes import *


In [ ]:
# Use air resistance or not?
air_resistance = False
    
# Maximum number of steps
n_steps = 1000

# Time step
tau = 0.05

# Max time
tmax = tau * n_steps
t = np.arange(0,tmax,tau)

#* Physical parameters (mass, Cd, etc.)
Cd = 0.35      # Drag coefficient (dimensionless)
area = 4.3e-3  # Cross-sectional area of projectile (m^2)
g = -9.81    # Gravitational acceleration (m/s^2)
mass = 0.145   # Mass of projectile (kg)
if not air_resistance :
    rho = 0      # No air resistance
else :
    rho = 1.2    # Density of air (kg/m^3)
air_const = 0.5*Cd*rho*area/mass  # Air resistance constant
print(f"Air resistance constant = {air_const}")

# Initial parameters
x0 = np.array([0., 1.8]).T
v0 = np.array([40., 0.]).T
a0 = np.array([0., g]).T

# Solutions
x = {}
v = {}
a = {}

### Calculate true solution with no air resistance

In [ ]:
x[ODEMethod.EXACT] = x0[:, None] + v0[:, None] * t + 0.5 * a0[:, None] * t**2

cut = (x[ODEMethod.EXACT][1, :] > 0)
print("Exact result with no air resistance:")
print(x[ODEMethod.EXACT][:, cut])
print("Output shape:")
print(x[ODEMethod.EXACT].shape)

### Simulate the trajectory

In [ ]:
#* Loop until ball hits ground or max steps completed

for method in [ODEMethod.EULER]: # ODEMethod.EULERCROMER, ODEMethod.MIDPOINT
    print(method.name)
    x[method] = np.zeros((2, n_steps))
    v[method] = np.zeros((2, n_steps))
    a[method] = np.zeros((2, n_steps))

    x[method][:, 0] = x0
    v[method][:, 0] = v0
    a[method][:, 0] = a0

    state = np.array([x[method][:, 0], v[method][:, 0], a[method][:, 0]])
    for i in np.arange(1, n_steps):
        t_i = t[i-1]
        abs_v = np.linalg.norm(state[1]) # speed
        state[2] = -air_const * abs_v * state[1]
        state[2][1] += g

        # New x and v
        if method == ODEMethod.EULER :
            state = step_euler(tau, state)
        elif method == ODEMethod.EULERCROMER :
            state = step_euler_cromer(tau, state)
        else : # Midpoint step
            state = step_midpoint(tau,state)        

        x[method][:, i] = state[0]
        v[method][:, i] = state[1]
        a[method][:, i] = state[2]


### Plot it for $y>0$

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for method in [ODEMethod.EXACT, ODEMethod.EULER]: # ODEMethod.EULERCROMER, ODEMethod.MIDPOINT
    cut = x[method][1, :] > 0
    if method == ODEMethod.EXACT:
        ls = "solid"
    else:
        ls = "dashed"
    ax.plot(x[method][0, cut], x[method][1, cut], label=method.name, ls=ls)
ax.legend()
